In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt

# Define network structure

In [2]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Prepare data

In [3]:
filename_input  = "input_1000.csv"
filename_output = "output_1000.csv"

x_train = np.genfromtxt(filename_input, delimiter=',')
y_train = np.genfromtxt(filename_output, delimiter=',')

# abnormal data transformation
y_train[:,0][y_train[:,0] > 0.5]  = 1       # rocof_max has been transformed to absolute value
y_train[:,1][y_train[:,1] < -1]   = -1      # fnadir
y_train[:,2][y_train[:,2] > 3.14] = 5       # maximum angle difference
y_train[:,3][y_train[:,3] > 1]    = 0.5     # maximum eigenvalue before gen trip

In [4]:
# check transformation results
np.max(y_train[:,0])

1.0

In [5]:
y_train[:,0]

## the generated dataset has some problems

array([0.0790901 , 0.0837153 , 0.0693797 , 1.        , 0.09572213,
       0.069409  , 1.        , 0.07524706, 0.08115491, 0.08216409,
       0.07967081, 0.06700474, 1.        , 0.08205921, 0.07585642,
       0.07130327, 0.08521941, 0.07579609, 1.        , 1.        ,
       1.        , 0.08072688, 0.07851557, 1.        , 0.08127209,
       1.        , 0.07004598, 0.079185  , 0.06856435, 0.08089331,
       1.        , 1.        , 0.06851926, 1.        , 0.08465932,
       0.08662053, 0.08314304, 0.07737936, 0.07120786, 0.06996988,
       0.08908645, 0.08302152, 0.07703347, 0.0794774 , 0.0816252 ,
       1.        , 0.08355392, 0.08205446, 0.07794086, 1.        ,
       0.07638384, 0.08132081, 0.08178905, 0.07695467, 0.07712435,
       0.07571843, 0.07456545, 1.        , 0.08117802, 0.08312721,
       0.07357502, 1.        , 1.        , 0.06735357, 0.06853761,
       0.07113582, 1.        , 0.07255603, 0.0743333 , 0.08712328,
       0.21655725, 0.06669274, 0.07412185, 0.0901608 , 0.07415

In [6]:

# convert to torch tensor for training
x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).float()

# Define training parameters

In [7]:
input_dim = 16 
hidden_dim = 128 
output_dim = 4

batch_size = 12

model = NeuralNetwork(input_dim, hidden_dim, output_dim)

dataset = TensorDataset(x_train, y_train)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [8]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

In [9]:
num_epochs = 3500  # Adjust as needed

for epoch in range(num_epochs):
    for batch_x, batch_y in dataloader:
        # Forward pass
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print training loss
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/3500], Loss: 0.0319
Epoch [2/3500], Loss: 0.0712
Epoch [3/3500], Loss: 0.0074
Epoch [4/3500], Loss: 0.0294
Epoch [5/3500], Loss: 0.1097
Epoch [6/3500], Loss: 0.0687
Epoch [7/3500], Loss: 0.0200
Epoch [8/3500], Loss: 0.0632
Epoch [9/3500], Loss: 0.0284
Epoch [10/3500], Loss: 0.0306
Epoch [11/3500], Loss: 0.0273
Epoch [12/3500], Loss: 1.3586
Epoch [13/3500], Loss: 0.0483
Epoch [14/3500], Loss: 0.0076
Epoch [15/3500], Loss: 0.0291
Epoch [16/3500], Loss: 0.0158
Epoch [17/3500], Loss: 0.0324
Epoch [18/3500], Loss: 0.0601
Epoch [19/3500], Loss: 0.0361
Epoch [20/3500], Loss: 0.0475
Epoch [21/3500], Loss: 0.0221
Epoch [22/3500], Loss: 0.0090
Epoch [23/3500], Loss: 0.0045
Epoch [24/3500], Loss: 0.0051
Epoch [25/3500], Loss: 0.0325
Epoch [26/3500], Loss: 0.0072
Epoch [27/3500], Loss: 0.0106
Epoch [28/3500], Loss: 0.0654
Epoch [29/3500], Loss: 0.0307
Epoch [30/3500], Loss: 0.0398
Epoch [31/3500], Loss: 0.0305
Epoch [32/3500], Loss: 0.0499
Epoch [33/3500], Loss: 0.0135
Epoch [34/3500], Lo

In [10]:
x1 = np.ones((16))*4
x1

array([4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.])

In [11]:
x1 = torch.from_numpy(x1).float()
x1

tensor([4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.])

In [12]:
model.forward(x1)

tensor([ 0.0039, -0.0300,  0.0433, -0.0074], grad_fn=<AddBackward0>)

# Save torch model

In [13]:
torch.save(model.state_dict(), 'trained_model.pth')

# Load saved model

In [14]:
# Load the saved model
loaded_model = NeuralNetwork(input_dim, hidden_dim, output_dim)  # Instantiate the model
loaded_model.load_state_dict(torch.load('trained_model.pth'))


<All keys matched successfully>

In [15]:
loaded_model.eval()  # Set the model to evaluation mode

NeuralNetwork(
  (fc1): Linear(in_features=16, out_features=128, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=128, out_features=4, bias=True)
)